In [1]:
import xarray as xr
import math
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import torch.nn as nn


In [2]:
import torch
import gpytorch
import matplotlib.pyplot as plt
import cf_units
import datetime
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [3]:
sea_level_dataset = xr.open_zarr('/scratch/aeb783/data/project/sea_level_picontrol_rechunked.zarr')
sea_level_dataarray = sea_level_dataset['sea_level']
#yt = xr.open_zarr('/scratch/aeb783/data/project/sea_level_picontrol_rechunked.zarr')['lat'].data
#xt = xr.open_zarr('/scratch/aeb783/data/project/sea_level_picontrol_rechunked.zarr')['long'].data
sea_level_dataarray

<xarray.DataArray 'sea_level' (time: 5844, yt_ocean: 2700, xt_ocean: 3600)>
dask.array<xarray-sea_level, shape=(5844, 2700, 3600), dtype=float32, chunksize=(5844, 27, 36), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) object 0185-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98

In [10]:
interval1 = sea_level_dataarray.sel(xt_ocean= -150, yt_ocean= 20, method="nearest").isel(time = slice(0,3000))

#This interval is half of the original data set, will be the training data output, y

timeint1 = interval1['time']

#Interval.plot()
# timearray1int = np.array(timeint1)
# timearray11 = cf_units.date2num(timearray1int,'days since 0185-01-01 12:00:00' , calendar='julian')


# #need to convert to nparray to then become a torch tensor
interval11 = np.array(interval1)

seaarray_torch = torch.FloatTensor(interval11).unsqueeze(dim=1)



new =seaarray_torch.reshape(100,30,1)


# timearray_torch = torch.FloatTensor(timearray11).squeeze()

# x_new = torch.cat((timearray_torch[0:200], timearray_torch[250:500], timearray_torch[1000:1600],timearray_torch[2600:4600]))

# y_new = torch.cat((seaarray_torch[0:200], seaarray_torch[250:500], seaarray_torch[1000:1600], seaarray_torch[2600:4600]))
# print(x_new)
# print(y_new)

print(new.shape)

torch.Size([100, 30, 1])


In [11]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(new)

seq_length = 4
x, y = sliding_windows(new, seq_length)

train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

In [6]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [7]:
num_epochs = 2000
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

RuntimeError: input must have 3 dimensions, got 1